# Mapping the Allen Mouse 3D Atlas to individual sections

This notebook uses the deformation fields and affine parameters to transform the annotations from the reference space to a specimen's image volume.

In [1]:
import numpy as np
import pandas as pd
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache, MouseConnectivityApi
from allensdk.api.queries.image_download_api import ImageDownloadApi
import SimpleITK as sitk
from matplotlib import pyplot as plt
from PIL import Image
import json
import nrrd
import scipy.ndimage as ndi
%matplotlib notebook

## Example IDs

For this notebook, this process will be applied to a section from a sample specimen.

In [2]:
ID = 100141219 # Image series ID for an experiment
section_no = 62

## Atlas and Image Data

The mouse connectivity cache is used to acquire the necessary atlas and image data.

In [3]:
mcc = MouseConnectivityCache(resolution=25,manifest_file='./mouse_connectivity_manifest.json')

### Get Atlas

In [4]:
anno,meta = mcc.get_annotation_volume()

In [5]:
rsp = mcc.get_reference_space()
rsp.remove_unassigned(); # This removes ids that are not in this particular reference space

In [6]:
name_map = rsp.structure_tree.get_name_map() # dictionary mapping ids to structure names

In [7]:
acrnm_map = rsp.structure_tree.get_id_acronym_map() # dictionary mapping acronyms to ids

In [8]:
colormap = rsp.structure_tree.get_colormap() # the colormap used for the allen 3D mouse atlas ontology

### Get structure ids for regions of interest

In [14]:
def get_structure_ids(acrnms):
    regions = []
    for acrnm in acrnms:
        regions.append(acrnm_map[acrnm])
    return regions

In [17]:
regions_of_interest = get_structure_ids(['GENd','GENv','LP','POL','APN'])

In [18]:
regions_of_interest

[1008, 1014, 218, 1029, 215]

## Get and set transform parameters

The registration of an image stack to the reference space involves a global (affine) registration followed by a local deformation. To map the atlas back onto the image space, we must first reverse the deformation and then the affine transformation. Thus, we acquire the deformation field and affine transform parameters and build the reverse transforms with SimpleITK.

In [20]:
# Get deformation field and set transform
dfmfld = mcc.get_deformation_field(ID,header_path='./{}/dfmfld.mhd'.format(ID),voxel_path='./{}/dfmfld.raw'.format(ID))
temp = sitk.ReadImage('./{}/dfmfld.mhd'.format(ID),sitk.sitkVectorFloat64)
dfmfld_transform = sitk.DisplacementFieldTransform(temp)

# Get affine parameters and set transform
temp = mcc.get_affine_parameters(ID,direction='trv',file_name='./{}/aff_param.txt'.format(ID))
aff_trans = sitk.AffineTransform(3)
aff_trans.SetParameters(temp.flatten())

## Get image section and associated data

Use the mouse connectivity api and image download api to download the image section of interest.

In [23]:
mca = MouseConnectivityApi()
ida = ImageDownloadApi()

# Get equalization values
details = mca.get_experiment_detail(ID)
equal = pd.DataFrame(details).T.loc['equalization'].values[0]
r = [equal['red_lower'],equal['red_upper'],equal['green_lower'],equal['green_upper'],equal['blue_lower'],equal['blue_upper']]
# Get 2D resolution
twoD = 1.0/details[0]['sub_images'][0]['resolution']
size = 25*twoD/(2**4)

# Get section id
sections_frame = pd.DataFrame(ida.section_image_query(ID))
section_id = sections_frame[sections_frame.section_number==section_no].id.values[0]

# Download image
ida.download_image(section_id,downsample=4,range=r,file_path='./{0}/{0}.jpg'.format(ID))

# Load image
img = Image.open('./{0}/{0}.jpg'.format(ID))
arr = np.array(img)

2019-09-04 09:49:53,867 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/image_download/102127598?downsample=4&range=0,911,0,975,0,4095


## Identify atlas sections in image section

The first step is to identify the sections in the atlas that map to the image section of interest. To do this, a mask of the entire brain is generated from which we get all the points within the brain reference space.

In [19]:
root_mask = rsp.make_structure_mask([997])
root_points = np.array(np.where(root_mask)).T.astype(float)

In [47]:
# Find all structures in section
# Transform root points
dfp = np.array(list(map(dfmfld_transform.TransformPoint,root_points*25)))
tfp = np.array(list(map(aff_trans.TransformPoint,dfp)))
# Find atlas slices corresponding to this section
sections = np.round(tfp[:,2]/100)
loc = np.where(sections == section_no)
# Find annotated ids in these slices
anno_loc = np.unique(root_points[loc][:,0])
structures_in_section = np.unique(anno[anno_loc.astype(int),:,:])[1:]

Once all the structures in the image section are found, the specific regions that may not appear in the annotated volumes, such as parent structures, can be added to the list of structures in the section. These are then sorted so that parent structures don't overlap children.

In [48]:
for region in regions:
    if region not in structures_in_section:
        structures_in_section = np.hstack((structures_in_section,region))
# Sort structure ids by ontology
sorted_indx = np.argsort(np.array(list(map(lambda x:len(x),rsp.structure_tree.ancestor_ids(structures_in_section)))))

## Map structures to image section

At this point, we can begin mapping the structures to this image section. We create an area_image to map the structure areas to, and a boundary image to map the desired region boundaries to.

In [49]:
area_image = arr.copy()
boundary_image = arr.copy()

In [50]:
for num,struct in enumerate(structures_in_section[sorted_indx]):
    if struct != 997: # There is no need to map the root structure here, since this was calculated previously
        # For each structure, the points (voxels) within the volume are acquired.
        mask = rsp.make_structure_mask([struct])
        points = np.array(np.where(mask)).T.astype(float)
        
        # These points are transformed using the deformation and affine transforms.
        dfp = np.array(list(map(dfmfld_transform.TransformPoint,points*25)))
        tfp = np.array(list(map(aff_trans.TransformPoint,dfp)))
        
        # Since these points will map to several sections, only the points in our desired section are selected
        sections = np.round(tfp[:,2]/100)
        loc = np.where(sections == section_no)
    
    # Map points to 2D section
    mapped_points = tfp[loc,:2]*twoD/(2**4)
    mapped_points = mapped_points.squeeze()
    next_points = mapped_points + size
    rounded_mapped_points,rounded_next_points = np.round(mapped_points).reshape((-1,2)),np.round(next_points).reshape((-1,2))
    
    # Place points on blank array and fill in gaps.
    blank = np.zeros((arr.shape[0],arr.shape[1]))
    for i,rounded_mapped_point in enumerate(rounded_mapped_points):
        rounded_next_point = rounded_next_points[i]
        rounded_mapped_point = rounded_mapped_point.astype(int)
        rounded_next_point = rounded_next_point.astype(int)
        blank[rounded_mapped_point[1]:rounded_next_point[1],rounded_mapped_point[0]:rounded_next_point[0]] = 1
    area = ndi.binary_closing(ndi.binary_fill_holes(blank).astype(int)).astype(int)
    
    # Place structre in area image
    color = colormap[struct]
    area_mask = np.where(area==1)
    for i in np.arange(3):
        area_image[:,:,i][area_mask] = color[i]
    
    # Draw region boundaries in boundary image
    if struct in regions:
        inner = area.copy()
        for i in np.arange(6):
            inner = ndi.binary_erosion(inner)
        boundary = area - inner.astype(int)
        boundary_mask = np.where(boundary == 1)
        for i in np.arange(3):
            boundary_image[:,:,i][boundary_mask] = 0
        boundary_image[:,:,2][boundary_mask] = 255

# Save area and boundary images        
area_img = Image.fromarray(area_image.astype(np.uint8))
area_img.save('./{}/area.png'.format(ID))
boundary_img = Image.fromarray(boundary_image.astype(np.uint8))
boundary_img.save('./{}/boundary.png'.format(ID))

# Create and save area composite
cbg = Image.open('./{0}/{0}.jpg'.format(ID))
fg = Image.open('./{}/area.png'.format(ID))
cbg_alpha = Image.new("L",cbg.size,255)
fg_alpha = Image.new("L",cbg.size,80)
cbg.putalpha(cbg_alpha)
fg.putalpha(fg_alpha)
comp = Image.alpha_composite(cbg,fg)
comp.save('./{}/color_composite.png'.format(ID))